In [1]:
import six
import sys
import time
import itertools
sys.modules['sklearn.externals.six'] = six
# import mlrose
import joblib
# sys.modules['sklearn.externals.joblib'] = j
import mlrose_hiive as mlrose
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

np.random.seed(42)
weights= np.random.uniform(low=0.1, high=1, size=(100,))
values= np.random.uniform(low=1, high=100, size=(100,))

G = nx.Graph()
    
cluster_sizes = [10, 10, 10]
base = 0
for size in cluster_sizes:
    cluster = nx.connected_watts_strogatz_graph(n=size, k=4, p=0.5, tries=100)
    cluster = nx.relabel_nodes(cluster, {i: i + base for i in range(size)})
    base += size
    G = nx.compose(G, cluster)
inter_cluster_edges = [(5, 15), (5, 25), (15, 20), (20, 25)]
G.add_edges_from(inter_cluster_edges)
edges = list(G.edges)


fitness_functions = [('Four Peaks', mlrose.FourPeaks()), ('Max K-Color', mlrose.MaxKColor(edges=edges)),\
                     ('Continuous Peaks', mlrose.ContinuousPeaks()),('K-Napsak', mlrose.Knapsack(weights, values))]
max_iterations = 5000
max_attempts = 20

### Randomized Hill Climbing - Restart tuning

In [2]:
best_rhc_parameter = None
best_rhc_fitness_value = None
restart_candinate =[0, 25, 50, 75, 100, 125, 150, 175, 200]

for fit_func in fitness_functions:
    function_name = fit_func[0]
    function = fit_func[1]

    print("Searchg Parameter for: ", function_name)
    best_rhc_parameter = None
    best_rhc_fitness_value = None
    
    for r_val in restart_candinate:
        problem_rhc = mlrose.DiscreteOpt(length=100, fitness_fn=function, maximize=True)
        rhc_best_state, rhc_best_fitness, rhc_fitness_curve = mlrose.random_hill_climb(problem_rhc, 
                                                                                     max_attempts = max_attempts, 
                                                                                  max_iters=max_iterations, 
                                                                                     curve=True, 
                                                                                     random_state=42,
                                                                                     restarts = r_val)

        if not best_rhc_fitness_value:
            best_rhc_parameter = r_val
            best_rhc_fitness_value = rhc_best_fitness
        elif rhc_best_fitness > best_rhc_fitness_value:
            best_rhc_parameter = r_val
            best_rhc_fitness_value = rhc_best_fitness

    print(f"Optimal Paratmeter for {function_name}: {best_rhc_parameter}")
    print()

Searchg Parameter for:  Four Peaks
Optimal Paratmeter for Four Peaks: 200

Searchg Parameter for:  Max K-Color
Optimal Paratmeter for Max K-Color: 100

Searchg Parameter for:  Continuous Peaks
Optimal Paratmeter for Continuous Peaks: 75

Searchg Parameter for:  K-Napsak
Optimal Paratmeter for K-Napsak: 25



### Simulated Annealing - init_temp, decay, min_temp

In [3]:
best_sa_parameter = None
best_sa_fitness_value = None
best_sa_decay = None
sa_parameter_candidate = [
    [1, 2, 4, 8, 16, 32, 64], #init_temp
   [0.1, 0.2, 0.4, 0.8], #decay
   [0.001, 0.01, 0.1, 1] #min_temp, exp_const
]


for fit_func in fitness_functions:
    function_name = fit_func[0]
    function = fit_func[1]

    print("Searchg Parameter for: ", function_name)
    best_sa_parameter = None
    best_sa_fitness_value = None
    best_sa_decay = None
    
    for sa_hyperparameter in itertools.product(*sa_parameter_candidate):
        problem_sa = mlrose.DiscreteOpt(length=100, fitness_fn=function, maximize=True)
        sa_decay_candidate = [("GeomDecay",mlrose.GeomDecay(init_temp=sa_hyperparameter[0], decay=sa_hyperparameter[1], min_temp=sa_hyperparameter[2])),\
                 ("ExpDecay",mlrose.ExpDecay(init_temp=sa_hyperparameter[0], exp_const=sa_hyperparameter[2]))]

        for d_val in sa_decay_candidate:                                                                                                                
            sa_best_state, sa_best_fitness, sa_fitness_curve = mlrose.simulated_annealing(problem_sa, 
                                                                                        max_attempts = max_attempts, 
                                                                                        max_iters=max_iterations, 
                                                                                        curve=True, 
                                                                                        random_state=42,
                                                                                        schedule = d_val[1])

            if not best_sa_fitness_value:
                best_sa_decay = d_val[0]
                best_sa_parameter = sa_hyperparameter
                best_sa_fitness_value = sa_best_fitness
            elif sa_best_fitness > best_sa_fitness_value:
                best_sa_decay = d_val[0]
                best_sa_parameter = sa_hyperparameter
                best_sa_fitness_value = sa_best_fitness

    if best_sa_decay == "ExpDecay":
        best_sa_parameter=(best_sa_parameter[0], best_sa_parameter[2])
    print(f"Optimal Paratmeter for {function_name}: {best_sa_parameter}, {best_sa_decay}")
    print()


Searchg Parameter for:  Four Peaks
Optimal Paratmeter for Four Peaks: (4, 0.001), ExpDecay

Searchg Parameter for:  Max K-Color
Optimal Paratmeter for Max K-Color: (4, 0.001), ExpDecay

Searchg Parameter for:  Continuous Peaks
Optimal Paratmeter for Continuous Peaks: (4, 0.001), ExpDecay

Searchg Parameter for:  K-Napsak
Optimal Paratmeter for K-Napsak: (64, 0.001), ExpDecay



### Genetic Algorithm - pop_size, mutation_prob

In [4]:
best_ga_parameter = None
best_ga_fitness_value = None
ga_parameter_candidate = [
    [100, 200, 400, 600], # pop_size
   [0.2, 0.4, 0.5, 0.6,0.8] #mutation_prob
]

for fit_func in fitness_functions:
    function_name = fit_func[0]
    function = fit_func[1]

    print("Searchg Parameter for: ", function_name)
    best_ga_parameter = None
    best_ga_fitness_value = None

    for ga_hyperparameter in itertools.product(*ga_parameter_candidate):
        problem_ga = mlrose.DiscreteOpt(length=100, fitness_fn=function, maximize=True)
        ga_best_state, ga_best_fitness, ga_fitness_curve = mlrose.genetic_alg(
                                                    problem_ga, 
                                                    max_attempts=max_attempts, 
                                                    max_iters=max_iterations, 
                                                    curve=True, 
                                                    random_state=42,
                                                    pop_size=ga_hyperparameter[0],
                                                    mutation_prob=ga_hyperparameter[1])
        if not best_ga_fitness_value:
            best_ga_parameter = ga_hyperparameter
            best_ga_fitness_value = ga_best_fitness
        elif ga_best_fitness > best_ga_fitness_value:
            best_ga_parameter = ga_hyperparameter
            best_ga_fitness_value = ga_best_fitness

    print(f"Optimal Paratmeter for {function_name}: {best_ga_parameter}")
    print()

Searchg Parameter for:  Four Peaks
Optimal Paratmeter for Four Peaks: (100, 0.8)

Searchg Parameter for:  Max K-Color
Optimal Paratmeter for Max K-Color: (100, 0.6)

Searchg Parameter for:  Continuous Peaks
Optimal Paratmeter for Continuous Peaks: (200, 0.4)

Searchg Parameter for:  K-Napsak
Optimal Paratmeter for K-Napsak: (600, 0.2)



### Mimic - keep_pct

In [5]:
best_mm_parameter = None
best_mm_fitness_value = None
keep_pct_candinate =[0.1, 0.25, 0.4, 0.5, 0.75]

for fit_func in fitness_functions:
    function_name = fit_func[0]
    function = fit_func[1]

    print("Searchg Parameter for: ", function_name)
    best_mm_parameter = None
    best_mm_fitness_value = None
    
    for k_val in keep_pct_candinate:
        problem_mm = mlrose.DiscreteOpt(length=100, fitness_fn=function, maximize=True)
        mm_best_state, mm_best_fitness, mm_fitness_curve = mlrose.mimic(problem_mm, 
                                                                                     max_attempts=100, 
                                                                                     max_iters=100, 
                                                                                     curve=True, 
                                                                                     random_state=42,
                                                                                     keep_pct=k_val)

        if not best_mm_fitness_value:
            best_mm_parameter = k_val
            best_mm_fitness_value = mm_best_fitness
        elif mm_best_fitness > best_mm_fitness_value:
            best_mm_parameter = k_val
            best_mm_fitness_value = mm_best_fitness

    print(f"Optimal Paratmeter for {function_name}: {best_mm_parameter}")
    print()

Searchg Parameter for:  Four Peaks
Optimal Paratmeter for Four Peaks: 0.1

Searchg Parameter for:  Max K-Color
Optimal Paratmeter for Max K-Color: 0.1

Searchg Parameter for:  Continuous Peaks
Optimal Paratmeter for Continuous Peaks: 0.25

Searchg Parameter for:  K-Napsak
Optimal Paratmeter for K-Napsak: 0.1

